# Chapter 3: Models
In this chapter, we explain the ideas behind model and agent interfaces and the example model/agent implementations provided by `cognibench`.

## Capabilities
Similar to `sciunit`, `cognibench` test classes require models to have certain capabilities. For example, `InteractiveTest` requires the models to implement the requirements specified by `cognibench.capabilities.Interactive` capability. In addition to test classes, the score class used for testing can also require capabilities. If you want to use `cognibench.scores.AICScore`, then the model should also have `cognibench.capabilities.PredictsLogpdf` and `cognibench.capabilities.ReturnsNumParams` capabilities. We can see these requirements as follows:

In [1]:
from cognibench.testing import InteractiveTest
from cognibench.scores import AICScore
print(InteractiveTest.required_capabilities)
print(AICScore.required_capabilities)

(<class 'cognibench.capabilities.Interactive'>,)
(<class 'cognibench.capabilities.PredictsLogpdf'>, <class 'cognibench.capabilities.ReturnsNumParams'>)


## Model Interface
In `cognibench` terms, a model represents a way of predicting the actions of some agent and fitting parameters to data:

In [2]:
from cognibench.models import CNBModel
print('CNBModel Documentation')
print('----------------------')
print(CNBModel.__doc__)

CNBModel Documentation
----------------------

    Base class for CogniBench models.

    In `cognibench`, a model is a way of representing a continuum of an agent and corresponding parameters. Models are
    expected to provide fitting and action prediction functionalities, while leaving the tasks of acting on environments
    and updating hidden state variables to agents. However, this distinction is not strictly enforced, and users may
    choose to use models to represent both of these concepts together.
    


## Agent Interface
An agent represents a way of interacting with an environment through `act` and `update` methods:

In [3]:
from cognibench.models import CNBAgent
print('CNBAgent Documentation')
print('----------------------')
print(CNBAgent.__doc__)

CNBAgent Documentation
----------------------

    Base class for CogniBench agents.

    In `cognibench`, an agent is a way of interacting with environments through `act` and `update` methods while possibly
    storing some hidden state.
    


## Example Agents and Models
In this section, we go over example model and agent implementations provided by cognibench:

In [4]:
# import Rescorla-Wagner Choice Kernel model
from cognibench.models.decision_making import RWCKModel, RWCKAgent

If we look at the superclasses of `RWCKAgent`, we see that it inherits from

In [5]:
RWCKAgent.__bases__

(cognibench.models.base.CNBAgent,
 cognibench.capabilities.ProducesPolicy,
 cognibench.capabilities.DiscreteAction,
 cognibench.capabilities.DiscreteObservation)

Here, the last two define the action and observation space this agent can interact with. This is important for methods that require environment and agent interaction such as `cognibench.simulation.simulate` function. Then, we see that `RWCKAgent` inherits from `ProducesPolicy` capability. This means that `RWCKAgent` provides `eval_policy` method that returns a probability distribution over the action space. Although this is not directly required for interacting with environments, it allows a very straightforward model implementation. In `cognibench`, this functionality is in `cognibench.models.policy_model.PolicyModel` class.

In [6]:
from cognibench.models.policy_model import PolicyModel
print(PolicyModel.__doc__)


    PolicyModel provides a model implementation that can be created from agents satisfying
    :class:`cognibench.capabilities.ProducesPolicy` capability.

    If you already have an agent implementation that can provide a probability distribution over the action space
    (`eval_policy` method), you can create a model of that agent that uses `eval_policy` to make predictions and to fit
    model parameters (using maximum likelihood) by simply deriving from this class. For examples of such usage, refer to
    decision making or associative learning model implementations provided by `cognibench`.
    


 Any model that subclass from `PolicyModel` automatically gets `fit` and `predict` method implementations by using the `eval_policy` method of the underlying agent. `RWCKModel` is an example of such implementation; it subclasses from `PolicyModel` and uses an `RWCKAgent` for the underlying update equations and probability distribution generation.

In [7]:
RWCKModel.__bases__

(cognibench.models.policy_model.PolicyModel,
 cognibench.capabilities.DiscreteAction,
 cognibench.capabilities.DiscreteObservation)

## Implementing Your Own Agents or Models
You can implement models by taking any of the examples as a starting point. For interactive tests, satisfying `cognibench.capabilities.Interactive` is a requirement. Apart from this, models can continuously satisfy new capabilities in order to take new tests.